In [ ]:
# MemorySafe Labs — Taste Demo v2
import os

REPO_URL = "https://github.com/MemorySafe-Labs/memorysafe.git"
REPO_DIR = "/content/memorysafe"

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL} {REPO_DIR}
else:
    print("Repo already exists:", REPO_DIR)

os.chdir(REPO_DIR)
print("CWD:", os.getcwd())

!pip -q install numpy matplotlib


In [ ]:
import os
demo_dir = "benchmarks/Taste_demo_v2"
assert os.path.isdir(demo_dir), f"Missing {demo_dir}"
print("Demo files:", os.listdir(demo_dir))


In [ ]:
import os, time
os.makedirs("benchmarks/Taste_demo_v2/_runs", exist_ok=True)

STEPS = 800
CAPACITY = 500
SEED = 7

def run_and_save(policy):
    outfile = f"benchmarks/Taste_demo_v2/_runs/{policy}_steps{STEPS}_cap{CAPACITY}_seed{SEED}.jsonl"
    cmd = f"python benchmarks/Taste_demo_v2/simulate_stream.py --policy {policy} --steps {STEPS} --capacity {CAPACITY} --seed {SEED}"
    print("Running:", cmd)
    print("Saving to:", outfile)

    !bash -lc "{cmd} | tee {outfile}"
    return outfile

logs = {}
for p in ["fifo", "reservoir", "random_replay", "memorysafe"]:
    logs[p] = run_and_save(p)

logs


In [ ]:
!ls -lah benchmarks/Taste_demo_v2/_runs


In [ ]:
import subprocess, sys

logfile = logs["memorysafe"]
cmd = [sys.executable, "benchmarks/Taste_demo_v2/plot_governance.py", "--logfile", logfile]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=False)


In [ ]:
!python benchmarks/Taste_demo_v2/plot_governance.py --help


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re

def extract_metric(path):
    vals = []
    with open(path) as f:
        for line in f:
            m = re.search(r"mvi_mean=([0-9.]+)", line)
            if m:
                vals.append(float(m.group(1)))
    return vals

fifo_vals = extract_metric(logs["fifo"])
mem_vals = extract_metric(logs["memorysafe"])

n = min(len(fifo_vals), len(mem_vals))
x = np.arange(n)

plt.figure()
plt.plot(x, fifo_vals[:n], label="FIFO")
plt.plot(x, mem_vals[:n], label="MemorySafe")
plt.title("Memory Vulnerability Governance")
plt.xlabel("Training steps")
plt.ylabel("MVI mean")
plt.legend()
plt.show()


In [ ]:
print("✅ Demo finished. (Run-all ready)")
